# Topic Relevant Results

In [ ]:
import re
import json
import pandas as pd
import numpy as np
from ast import literal_eval


with open('dataset/ideabench_clustering.json', 'r') as f:
    data = json.load(f)

tabdf = pd.DataFrame({'cluster':list(data.keys())})

def calculate_mean_median(bert_scores, llm_ratings):
    mean_bert_scores = round(bert_scores['f1'].mean(), 3)
    median_bert_scores = round(bert_scores['f1'].median(), 3)
    percentile_80_bert_scores = round(bert_scores['f1'].quantile(0.8), 3)
    
    mean_llm_rating = round(llm_ratings.mean(), 3)
    median_llm_rating = round(llm_ratings.median(), 3)
    percentile_80_llm_rating = round(llm_ratings.quantile(0.8), 3)
    
    return mean_bert_scores, median_bert_scores, percentile_80_bert_scores, mean_llm_rating, median_llm_rating, percentile_80_llm_rating

def extract_llm_rating(rating_str):
    match = re.search(r'\d+', rating_str)
    if match:
        return int(match.group())
    else:
        return None

def extract_metrics(df):
    bert_scores = {'precision': [], 'recall': [], 'f1': []}
    llm_ratings = []
    
    for idx in range(len(df)):
        try:
            metrics = literal_eval(df['metrics'].iloc[idx])
            bert_scores['precision'].append(metrics['bert_score']['precision'])
            bert_scores['recall'].append(metrics['bert_score']['recall'])
            bert_scores['f1'].append(metrics['bert_score']['f1'])
            llm_rating = extract_llm_rating(metrics['llm_evaluation']['rating'])
            if llm_rating is not None:
                llm_ratings.append(llm_rating)
            else:
                llm_rating = extract_llm_rating(metrics['llm_evaluation']['explanation'])
                llm_ratings.append(llm_rating)
        except KeyError:
            continue
        
    return pd.DataFrame(bert_scores), pd.Series(llm_ratings)

def find_scores_for_model(dataframe):
    bert_scores, llm_ratings = extract_metrics(dataframe)
    
    mean_median_percentiles = calculate_mean_median(bert_scores, llm_ratings)
    
    return mean_median_percentiles

def extract_hypothesis_order(text):
    pattern = r'\d+\.\s\*\*Hypothesis ([A-Z])\*\*'
    matches = re.findall(pattern, text)
    return list(matches)

def update_ranking_list(row):
    lst, text = literal_eval(row)
    if not lst:
        return extract_hypothesis_order(text)
    return lst

def get_relative_rank(ranking):
    if "A" in ranking:
        numerator = ranking.index('A')
        denominator = len(ranking) - 1
        if denominator == 0:
            return 0
        rel_rank = numerator / denominator
        return rel_rank
    else:
        return None

def avg_rel_rank(rankings):
    relative_ranks = [get_relative_rank(row) for row in rankings]
    relative_ranks_array = np.array(relative_ranks)
    cleaned_array = relative_ranks_array[relative_ranks_array != None]

    mean = np.mean(cleaned_array)
    median = np.median(cleaned_array)
    # percentile_75th = np.percentile(cleaned_array, 75)

    return mean




In [ ]:
names = ['Llamma-3.1 405B High Resource', 'Llamma-3.1 405B Low Resource', 'Llamma-3.1 70B High Resource', 'Llamma-3.1 70B Low Resource', 'Gemini 1.5 Flash High Resource', 'Gemini 1.5 Flash Low Resource', 'Gemini 1.5 Pro High Resource', 'Gemini 1.5 Pro Low Resource', 'GPT-3.5 Turbo High Resource', 'GPT-3.5 Turbo Low Resource', 'GPT-4o Mini High Resource', 'GPT-4o Mini Low Resource', 'GPT-4o High Resource', 'GPT-4o Low Resource']
path = ['generated_hypotheses/gen_hyp_llama3.1-405b_all_ref_eval.csv', 'generated_hypotheses/gen_hyp_llama3.1-405b_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_llama3.1-70b_all_ref_eval.csv', 'generated_hypotheses/gen_hyp_llama3.1-70b_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_gemini-1.5-flash_all_ref_unfiltered_eval.csv', 'generated_hypotheses/gen_hyp_gemini-1.5-flash_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_gemini-1.5-pro_all_ref_unfiltered_eval.csv', '/generated_hypotheses/gen_hyp_gemini-1.5-pro_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_15_ref_eval.csv', 'generated_hypotheses/gen_hyp_gpt-3.5-turbo_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_gpt-4o-mini_all_ref_eval.csv', 'generated_hypotheses/gen_hyp_gpt-4o-mini_5_filtered_ref_eval.csv', 'generated_hypotheses/gen_hyp_gpt-4o_all_ref_eval.csv', 'generated_hypotheses/gen_hyp_gpt-4o_5_filtered_ref_eval.csv']


In [ ]:
for high in range(0, len(names), 2):
    print(high)
    low = high + 1
    high_df = pd.read_csv(path[high])
    low_df = pd.read_csv(path[low])
    curr_map = {'cluster':[], f'{names[high]} - Insight Score (feasibility)':[], f'{names[low]} - Insight Score (feasibility)':[], f'{names[high]} - Insight Score (novelty)':[], f'{names[low]} - Insight Score (novelty)':[],
                f'{names[high]} - Mean BERTScore':[], f'{names[high]} - Median BERTScore':[], f'{names[high]} - 80th Percentile BERTScore':[], f'{names[high]} - Mean LLM Rating':[], f'{names[high]} - Median LLM Rating':[], f'{names[high]} - 80th Percentile LLM Rating':[], f'{names[low]} - Mean BERTScore':[], f'{names[low]} - Median BERTScore':[], f'{names[low]} - 80th Percentile BERTScore':[], f'{names[low]} - Mean LLM Rating':[], f'{names[low]} - Median LLM Rating':[], f'{names[low]} - 80th Percentile LLM Rating':[]}
    for k in data:
        curr_map['cluster'].append(k)
        filtered_high_df = high_df[high_df["paperId"].isin(data[k])]
        filtered_low_df = low_df[low_df["paperId"].isin(data[k])]
        filtered_high_df['llm_novelty_ranking_eval'] = filtered_high_df['llm_novelty_ranking_eval'].apply(update_ranking_list)
        filtered_high_df['llm_feasibility_ranking_eval'] = filtered_high_df['llm_feasibility_ranking_eval'].apply(update_ranking_list)
        filtered_low_df['llm_novelty_ranking_eval'] = filtered_low_df['llm_novelty_ranking_eval'].apply(update_ranking_list)
        filtered_low_df['llm_feasibility_ranking_eval'] = filtered_low_df['llm_feasibility_ranking_eval'].apply(update_ranking_list)

        high_mean_median_percentiles = find_scores_for_model(filtered_high_df)
        low_mean_median_percentiles = find_scores_for_model(filtered_low_df)

        curr_map[f'{names[high]} - Mean BERTScore'].append(high_mean_median_percentiles[0])
        curr_map[f'{names[high]} - Median BERTScore'].append(high_mean_median_percentiles[1])
        curr_map[f'{names[high]} - 80th Percentile BERTScore'].append(high_mean_median_percentiles[2])
        curr_map[f'{names[high]} - Mean LLM Rating'].append(high_mean_median_percentiles[3])
        curr_map[f'{names[high]} - Median LLM Rating'].append(high_mean_median_percentiles[4])
        curr_map[f'{names[high]} - 80th Percentile LLM Rating'].append(high_mean_median_percentiles[5])

        curr_map[f'{names[low]} - Mean BERTScore'].append(low_mean_median_percentiles[0])
        curr_map[f'{names[low]} - Median BERTScore'].append(low_mean_median_percentiles[1])
        curr_map[f'{names[low]} - 80th Percentile BERTScore'].append(low_mean_median_percentiles[2])
        curr_map[f'{names[low]} - Mean LLM Rating'].append(low_mean_median_percentiles[3])
        curr_map[f'{names[low]} - Median LLM Rating'].append(low_mean_median_percentiles[4])
        curr_map[f'{names[low]} - 80th Percentile LLM Rating'].append(low_mean_median_percentiles[5])

        novelty_avg_rel_rank_high = avg_rel_rank(filtered_high_df['llm_novelty_ranking_eval'])
        feasibility_avg_rel_rank_high = avg_rel_rank(filtered_high_df['llm_feasibility_ranking_eval'])
        novelty_avg_rel_rank_low = avg_rel_rank(filtered_low_df['llm_novelty_ranking_eval'])
        feasibility_avg_rel_rank_low = avg_rel_rank(filtered_low_df['llm_feasibility_ranking_eval'])

        curr_map[f'{names[high]} - Insight Score (feasibility)'].append(feasibility_avg_rel_rank_high)
        curr_map[f'{names[low]} - Insight Score (feasibility)'].append(feasibility_avg_rel_rank_low)
        curr_map[f'{names[high]} - Insight Score (novelty)'].append(novelty_avg_rel_rank_high)
        curr_map[f'{names[low]} - Insight Score (novelty)'].append(novelty_avg_rel_rank_low)

    n_df = pd.DataFrame(curr_map)
    tabdf = pd.merge(tabdf, n_df, on='cluster', how='inner')

    tabdf.to_csv('ideabench_cluster_res.csv', index=False)

In [ ]:
import json

with open('dataset/ideabench_clustering.json', 'r') as file:
    cluster_stats = json.load(file)


# Get results for each topic

### BertScores

In [ ]:

cluster_results = pd.read_csv('ideabench_cluster_res.csv')


In [5]:
i=0
health_medicine_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]

# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(health_medicine_table):
    print(f"Row {row_labels[row_index]}:")
    print(len(row), len(column_labels))
    for label, value in zip(column_labels, row):
        print(f"{label}: {round(value, 3)}")
    print("-" * 40)  # Separator between rows

Row Llamma-3.1 70B Low Resource:
6 6
Mean BERTScore: 0.587
SD BERTScore: 0.058
Insight Score (novelty) Mean: 0.637
Insight Score (novelty) STD: 0.406
Insight Score (feasibility) Mean: 0.114
Insight Score (feasibility) STD: 0.259
----------------------------------------
Row Llamma-3.1 70B High Resource:
6 6
Mean BERTScore: 0.606
SD BERTScore: 0.056
Insight Score (novelty) Mean: 0.622
Insight Score (novelty) STD: 0.456
Insight Score (feasibility) Mean: 0.11
Insight Score (feasibility) STD: 0.291
----------------------------------------
Row Llamma-3.1 405B Low Resource:
6 6
Mean BERTScore: 0.565
SD BERTScore: 0.042
Insight Score (novelty) Mean: 0.674
Insight Score (novelty) STD: 0.348
Insight Score (feasibility) Mean: 0.096
Insight Score (feasibility) STD: 0.23
----------------------------------------
Row Llamma-3.1 405B High Resource:
6 6
Mean BERTScore: 0.586
SD BERTScore: 0.05
Insight Score (novelty) Mean: 0.703
Insight Score (novelty) STD: 0.348
Insight Score (feasibility) Mean: 0.1
I

In [6]:
i=1
genetics_molecular_biology_table =  [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(genetics_molecular_biology_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # Separator between rows

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.585
  SD BERTScore: 0.055
  Insight Score (novelty) Mean: 0.5954933126574915
  Insight Score (novelty) STD: 0.412
  Insight Score (feasibility) Mean: 0.1498459761262996
  Insight Score (feasibility) STD: 0.308
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.597
  SD BERTScore: 0.061
  Insight Score (novelty) Mean: 0.5469378574641734
  Insight Score (novelty) STD: 0.475
  Insight Score (feasibility) Mean: 0.136144849302744
  Insight Score (feasibility) STD: 0.32
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.561
  SD BERTScore: 0.044
  Insight Score (novelty) Mean: 0.6081835390025043
  Insight Score (novelty) STD: 0.372
  Insight Score (feasibility) Mean: 0.1293913819129957
  Insight Score (feasibility) STD: 0.254
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.585
  SD BERTScore: 0.049
  Insig

In [7]:
i=2
environmental_sciences_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(environmental_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.576
  SD BERTScore: 0.045
  Insight Score (novelty) Mean: 0.6388555422168868
  Insight Score (novelty) STD: 0.396
  Insight Score (feasibility) Mean: 0.169467401285583
  Insight Score (feasibility) STD: 0.309
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.581
  SD BERTScore: 0.064
  Insight Score (novelty) Mean: 0.5760057471264367
  Insight Score (novelty) STD: 0.467
  Insight Score (feasibility) Mean: 0.212121212121212
  Insight Score (feasibility) STD: 0.374
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.546
  SD BERTScore: 0.038
  Insight Score (novelty) Mean: 0.6142006802721091
  Insight Score (novelty) STD: 0.359
  Insight Score (feasibility) Mean: 0.156281887755102
  Insight Score (feasibility) STD: 0.267
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.561
  SD BERTScore: 0.053
  Insigh

In [8]:
i=3
neuroscience_cognitive_sciences_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(neuroscience_cognitive_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.586
  SD BERTScore: 0.056
  Insight Score (novelty) Mean: 0.621050164088139
  Insight Score (novelty) STD: 0.41
  Insight Score (feasibility) Mean: 0.1995175525258512
  Insight Score (feasibility) STD: 0.34
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.595
  SD BERTScore: 0.054
  Insight Score (novelty) Mean: 0.586076817558299
  Insight Score (novelty) STD: 0.463
  Insight Score (feasibility) Mean: 0.1689986282578875
  Insight Score (feasibility) STD: 0.336
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.556
  SD BERTScore: 0.044
  Insight Score (novelty) Mean: 0.6278581474784005
  Insight Score (novelty) STD: 0.372
  Insight Score (feasibility) Mean: 0.1541061339790153
  Insight Score (feasibility) STD: 0.288
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.582
  SD BERTScore: 0.049
  Insight

In [9]:
i=4
technology_engineering_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]



row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(technology_engineering_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.595
  SD BERTScore: 0.037
  Insight Score (novelty) Mean: 0.5477876106194691
  Insight Score (novelty) STD: 0.412
  Insight Score (feasibility) Mean: 0.1843447113358617
  Insight Score (feasibility) STD: 0.324
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.604
  SD BERTScore: 0.043
  Insight Score (novelty) Mean: 0.5821637426900585
  Insight Score (novelty) STD: 0.454
  Insight Score (feasibility) Mean: 0.2340016708437761
  Insight Score (feasibility) STD: 0.4
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.553
  SD BERTScore: 0.041
  Insight Score (novelty) Mean: 0.5805280528052805
  Insight Score (novelty) STD: 0.369
  Insight Score (feasibility) Mean: 0.2240226337448559
  Insight Score (feasibility) STD: 0.322
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.57
  SD BERTScore: 0.042
  Insigh

In [10]:
i=5
social_behavioral_sciences_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(social_behavioral_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.578
  SD BERTScore: 0.03
  Insight Score (novelty) Mean: 0.6641931216931216
  Insight Score (novelty) STD: 0.403
  Insight Score (feasibility) Mean: 0.2027380952380952
  Insight Score (feasibility) STD: 0.342
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.586
  SD BERTScore: 0.036
  Insight Score (novelty) Mean: 0.6776004256451187
  Insight Score (novelty) STD: 0.414
  Insight Score (feasibility) Mean: 0.1981216931216931
  Insight Score (feasibility) STD: 0.351
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.544
  SD BERTScore: 0.034
  Insight Score (novelty) Mean: 0.7044259421560034
  Insight Score (novelty) STD: 0.354
  Insight Score (feasibility) Mean: 0.1358917480035492
  Insight Score (feasibility) STD: 0.287
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.56
  SD BERTScore: 0.04
  Insigh

In [11]:
i=6
materials_science_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]
row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(materials_science_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.582
  SD BERTScore: 0.056
  Insight Score (novelty) Mean: 0.5221340388007054
  Insight Score (novelty) STD: 0.428
  Insight Score (feasibility) Mean: 0.1589098532494759
  Insight Score (feasibility) STD: 0.296
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.59
  SD BERTScore: 0.067
  Insight Score (novelty) Mean: 0.5732026143790849
  Insight Score (novelty) STD: 0.461
  Insight Score (feasibility) Mean: 0.2222222222222222
  Insight Score (feasibility) STD: 0.377
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.548
  SD BERTScore: 0.046
  Insight Score (novelty) Mean: 0.6352380952380954
  Insight Score (novelty) STD: 0.395
  Insight Score (feasibility) Mean: 0.2362745098039216
  Insight Score (feasibility) STD: 0.342
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.576
  SD BERTScore: 0.047
  Insi

In [12]:
i=7
public_health_policy_table =   [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility) STD']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility) STD']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty) STD'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility) STD']
    ]
]

# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean BERTScore',
    'SD BERTScore',
    'Insight Score (novelty) Mean', 
    'Insight Score (novelty) STD', 
    'Insight Score (feasibility) Mean',
    'Insight Score (feasibility) STD'
]
row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(public_health_policy_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean BERTScore: 0.59
  SD BERTScore: 0.037
  Insight Score (novelty) Mean: 0.7521934758155232
  Insight Score (novelty) STD: 0.335
  Insight Score (feasibility) Mean: 0.131266149870801
  Insight Score (feasibility) STD: 0.28
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean BERTScore: 0.598
  SD BERTScore: 0.045
  Insight Score (novelty) Mean: 0.7841823551125876
  Insight Score (novelty) STD: 0.367
  Insight Score (feasibility) Mean: 0.1718346253229974
  Insight Score (feasibility) STD: 0.352
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean BERTScore: 0.551
  SD BERTScore: 0.041
  Insight Score (novelty) Mean: 0.7707660455486545
  Insight Score (novelty) STD: 0.319
  Insight Score (feasibility) Mean: 0.1303535353535353
  Insight Score (feasibility) STD: 0.257
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean BERTScore: 0.574
  SD BERTScore: 0.048
  Insigh

# LLM Ratings

In [ ]:
# for item in cluster_stats:
#     print(item)
#     print(len(cluster_stats[item]))

cluster_results = pd.read_csv('idea')


In [14]:
cluster_results

,cluster,Llamma-3.1 405B High Resource - Insight Score (feasibility),Llamma-3.1 405B High Resource - Insight Score (feasibility) STD,Llamma-3.1 405B Low Resource - Insight Score (feasibility),Llamma-3.1 405B Low Resource - Insight Score (feasibility) STD,Llamma-3.1 405B High Resource - Insight Score (novelty),Llamma-3.1 405B High Resource - Insight Score (novelty) STD,Llamma-3.1 405B Low Resource - Insight Score (novelty),Llamma-3.1 405B Low Resource - Insight Score (novelty) STD,Llamma-3.1 405B High Resource - Mean BERTScore,...,GPT-4o Low Resource - 80th Percentile BERTScore,GPT-4o Low Resource - 25th Percentile BERTScore,GPT-4o Low Resource - 75th Percentile BERTScore,GPT-4o Low Resource - SD BERTScore,GPT-4o Low Resource - Mean LLM Rating,GPT-4o Low Resource - Median LLM Rating,GPT-4o Low Resource - 80th Percentile LLM Rating,GPT-4o Low Resource - 25th Percentile LLM Rating,GPT-4o Low Resource - 75th Percentile LLM Rating,GPT-4o Low Resource - SD LLM Rating
0,Health & Medicine,0.100167,0.228,0.095934,0.230,0.703280,0.348,0.673558,0.348,0.548,...,0.600,0.556,0.595,0.031,5.056,5.0,7.0,3.0,7.0,2.146
1,Genetics & Molecular Biology,0.134459,0.258,0.129391,0.254,0.679873,0.354,0.608184,0.372,0.555,...,0.603,0.564,0.599,0.028,5.017,5.0,7.0,3.0,7.0,2.096
2,Environmental Sciences,0.120269,0.218,0.156282,0.267,0.623451,0.368,0.614201,0.359,0.535,...,0.596,0.559,0.594,0.026,5.024,5.0,7.0,3.0,7.0,2.156
3,Neuroscience & Cognitive Sciences,0.161894,0.301,0.154106,0.288,0.631794,0.346,0.627858,0.372,0.554,...,0.595,0.560,0.592,0.025,5.310,5.0,8.0,3.0,7.0,2.100
4,Technology & Engineering,0.192229,0.323,0.224023,0.322,0.569164,0.334,0.580528,0.369,0.546,...,0.606,0.560,0.603,0.028,4.640,4.0,7.0,3.0,6.0,1.839
5,Social & Behavioral Sciences,0.171244,0.311,0.135892,0.287,0.629152,0.364,0.704426,0.354,0.535,...,0.586,0.551,0.582,0.025,4.950,5.0,7.0,3.0,7.0,1.992
6,Materials Science,0.250687,0.337,0.236275,0.342,0.722985,0.353,0.635238,0.395,0.549,...,0.601,0.559,0.595,0.027,4.818,4.0,7.2,3.0,6.0,2.170
7,Public Health & Policy,0.106266,0.230,0.130354,0.257,0.773688,0.312,0.770766,0.319,0.532,...,0.582,0.539,0.573,0.027,4.721,4.0,6.4,3.0,6.0,1.928


In [15]:
import numpy as np


In [16]:
i=0
health_medicine_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(health_medicine_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.886
  SD Idea Overlap: 2.371
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.654
  SD Idea Overlap: 2.291
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 5.132
  SD Idea Overlap: 2.381
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.613
  SD Idea Overlap: 2.352
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.629
  SD Idea Overlap: 2.241
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 5.45
  SD Idea Overlap: 2.279
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 4.27
  SD Idea Overlap: 2.122
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 5.183
  SD Idea Overlap: 2.188
-------------------------

In [17]:
i=1
genetics_molecular_biology_table  = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(genetics_molecular_biology_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.71
  SD Idea Overlap: 2.325
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.546
  SD Idea Overlap: 2.338
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 5.277
  SD Idea Overlap: 2.335
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.729
  SD Idea Overlap: 2.287
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.461
  SD Idea Overlap: 2.149
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 5.239
  SD Idea Overlap: 2.258
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 4.3
  SD Idea Overlap: 2.047
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 5.313
  SD Idea Overlap: 2.179
--------------------------

In [18]:
i=2
environmental_sciences_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(environmental_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.554
  SD Idea Overlap: 2.24
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.225
  SD Idea Overlap: 2.382
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 4.902
  SD Idea Overlap: 2.352
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.016
  SD Idea Overlap: 2.489
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.35
  SD Idea Overlap: 2.139
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 4.911
  SD Idea Overlap: 2.305
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 3.911
  SD Idea Overlap: 1.954
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 4.976
  SD Idea Overlap: 2.159
-------------------------

In [19]:
i=3
neuroscience_cognitive_sciences_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(neuroscience_cognitive_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 5.111
  SD Idea Overlap: 2.315
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.731
  SD Idea Overlap: 2.304
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 5.46
  SD Idea Overlap: 2.353
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.79
  SD Idea Overlap: 2.267
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.919
  SD Idea Overlap: 2.259
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 5.725
  SD Idea Overlap: 2.112
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 4.457
  SD Idea Overlap: 2.273
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 5.315
  SD Idea Overlap: 2.146
-------------------------

In [20]:
i=4
technology_engineering_table =  [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(technology_engineering_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.43
  SD Idea Overlap: 2.065
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 4.796
  SD Idea Overlap: 2.196
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 4.693
  SD Idea Overlap: 2.421
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.096
  SD Idea Overlap: 2.35
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.211
  SD Idea Overlap: 2.151
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 4.825
  SD Idea Overlap: 2.432
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 3.929
  SD Idea Overlap: 2.082
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 4.561
  SD Idea Overlap: 2.331
-------------------------

In [21]:
i=5
social_behavioral_sciences_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(social_behavioral_sciences_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.436
  SD Idea Overlap: 2.085
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.15
  SD Idea Overlap: 2.241
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 4.517
  SD Idea Overlap: 2.201
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.105
  SD Idea Overlap: 2.2
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.494
  SD Idea Overlap: 2.086
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 4.923
  SD Idea Overlap: 2.202
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 3.678
  SD Idea Overlap: 1.833
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 4.606
  SD Idea Overlap: 1.999
--------------------------

In [22]:
i=6
materials_science_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(materials_science_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.453
  SD Idea Overlap: 2.249
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 5.585
  SD Idea Overlap: 2.459
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 5.037
  SD Idea Overlap: 2.257
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.564
  SD Idea Overlap: 2.339
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.327
  SD Idea Overlap: 2.169
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 5.463
  SD Idea Overlap: 2.108
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 4.2
  SD Idea Overlap: 1.938
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 5.115
  SD Idea Overlap: 2.255
-------------------------

In [23]:
i=7
public_health_policy_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating']
    ]
]


# Define the column labels in the same order as the values in each row.
column_labels = [
    'Mean Idea Overlap',
    'SD Idea Overlap',
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(public_health_policy_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  Mean Idea Overlap: 4.299
  SD Idea Overlap: 2.087
----------------------------------------
Row Llamma-3.1 70B High Resource:
  Mean Idea Overlap: 4.716
  SD Idea Overlap: 2.302
----------------------------------------
Row Llamma-3.1 405B Low Resource:
  Mean Idea Overlap: 4.736
  SD Idea Overlap: 2.511
----------------------------------------
Row Llamma-3.1 405B High Resource:
  Mean Idea Overlap: 5.109
  SD Idea Overlap: 2.291
----------------------------------------
Row Gemini 1.5 Flash Low Resource:
  Mean Idea Overlap: 4.633
  SD Idea Overlap: 2.244
----------------------------------------
Row Gemini 1.5 Flash High Resource:
  Mean Idea Overlap: 5.147
  SD Idea Overlap: 2.201
----------------------------------------
Row Gemini 1.5 Pro Low Resource:
  Mean Idea Overlap: 4.234
  SD Idea Overlap: 2.094
----------------------------------------
Row Gemini 1.5 Pro High Resource:
  Mean Idea Overlap: 4.488
  SD Idea Overlap: 2.212
-----------------------

In [ ]:

cluster_results = pd.read_csv('ideabench_cluster_resv')


In [27]:
# similarity for all topic clusters
cluster_results.iloc[:]['Llamma-3.1 70B Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Llamma-3.1 70B High Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Llamma-3.1 405B Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Llamma-3.1 405B High Resource - 80th Percentile BERTScore']

cluster_results.iloc[:]['Gemini 1.5 Flash Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Gemini 1.5 Flash High Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Gemini 1.5 Pro Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['Gemini 1.5 Pro High Resource - 80th Percentile BERTScore']

cluster_results.iloc[:]['GPT-3.5 Turbo Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['GPT-4o Mini Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['GPT-4o Mini High Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['GPT-4o Low Resource - 80th Percentile BERTScore']
cluster_results.iloc[:]['GPT-4o High Resource - 80th Percentile BERTScore']

0    0.609
1    0.602
2    0.601
Name: GPT-4o High Resource - 80th Percentile BERTScore, dtype: float64

In [28]:
# idea overlap for all topic clusters
cluster_results.iloc[:]['Llamma-3.1 70B Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Llamma-3.1 70B High Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Llamma-3.1 405B Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Llamma-3.1 405B High Resource - 80th Percentile LLM Rating']

cluster_results.iloc[:]['Gemini 1.5 Flash Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Gemini 1.5 Flash High Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Gemini 1.5 Pro Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['Gemini 1.5 Pro High Resource - 80th Percentile LLM Rating']

cluster_results.iloc[:]['GPT-3.5 Turbo Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['GPT-4o Mini Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['GPT-4o Mini High Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['GPT-4o Low Resource - 80th Percentile LLM Rating']
cluster_results.iloc[:]['GPT-4o High Resource - 80th Percentile LLM Rating']

0    8.0
1    8.0
2    7.0
Name: GPT-4o High Resource - 80th Percentile LLM Rating, dtype: float64

In [29]:
# novelty for all topic clusters
cluster_results.iloc[:]['Llamma-3.1 70B Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Llamma-3.1 70B High Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Llamma-3.1 405B Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Llamma-3.1 405B High Resource - Insight Score (novelty)']

cluster_results.iloc[:]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Gemini 1.5 Flash High Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['Gemini 1.5 Pro High Resource - Insight Score (novelty)']

cluster_results.iloc[:]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['GPT-3.5 Turbo High Resource - Insight Score (novelty)']
cluster_results.iloc[:]['GPT-4o Mini Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['GPT-4o Mini High Resource - Insight Score (novelty)']
cluster_results.iloc[:]['GPT-4o Low Resource - Insight Score (novelty)']
cluster_results.iloc[:]['GPT-4o High Resource - Insight Score (novelty)']

0    0.776659
1    0.697432
2    0.735432
Name: GPT-4o High Resource - Insight Score (novelty), dtype: float64

In [30]:
# feasibility for all topic clusters
cluster_results.iloc[:]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Llamma-3.1 70B High Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Llamma-3.1 405B High Resource - Insight Score (feasibility)']

cluster_results.iloc[:]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)']

cluster_results.iloc[:]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['GPT-4o Mini Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['GPT-4o Mini High Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['GPT-4o Low Resource - Insight Score (feasibility)']
cluster_results.iloc[:]['GPT-4o High Resource - Insight Score (feasibility)']

0    0.149946
1    0.221509
2    0.238418
Name: GPT-4o High Resource - Insight Score (feasibility), dtype: float64

In [31]:
print(cluster_results.iloc[0]['Llamma-3.1 70B Low Resource - 80th Percentile BERTScore']
)

0.589


In [33]:
i=0
life_health_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)']
    ]
]
life_health_table

# Define the column labels in the same order as the values in each row.
column_labels = [
    '25th Percentile BERTScore', 
    'Median BERTScore',
    'Mean BERTScore',
    'SD BERTScore',
    '75th Percentile BERTScore',
    '80th Percentile BERTScore',
    '25th Percentile LLM Rating',
    'Median LLM Rating',
    'Mean LLM Rating',
    'SD LLM Rating',
    '75th Percentile LLM Rating',
    '80th Percentile LLM Rating',
    'Insight Score (novelty)', 
    'Insight Score (feasibility)'
]

row_labels = [
    'Llamma-3.1 70B Low Resource',
    'Llamma-3.1 70B High Resource',
    'Llamma-3.1 405B Low Resource',
    'Llamma-3.1 405B High Resource',
    'Gemini 1.5 Flash Low Resource',
    'Gemini 1.5 Flash High Resource',
    'Gemini 1.5 Pro Low Resource',
    'Gemini 1.5 Pro High Resource',
    'GPT-3.5 Turbo Low Resource',
    'GPT-3.5 Turbo High Resource',
    'GPT-4o Mini Low Resource',
    'GPT-4o Mini High Resource',
    'GPT-4o Low Resource',
    'GPT-4o High Resource'
]

# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(life_health_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # Separator between rows

Row Llamma-3.1 70B Low Resource:
  25th Percentile BERTScore: 0.531
  Median BERTScore: 0.559
  Mean BERTScore: 0.547
  SD BERTScore: 0.057
  75th Percentile BERTScore: 0.583
  80th Percentile BERTScore: 0.589
  25th Percentile LLM Rating: 3.0
  Median LLM Rating: 4.0
  Mean LLM Rating: 4.805
  SD LLM Rating: 2.333
  75th Percentile LLM Rating: 7.0
  80th Percentile LLM Rating: 7.0
  Insight Score (novelty): 0.626369771279951
  Insight Score (feasibility): 0.1407012652155663
----------------------------------------
Row Llamma-3.1 70B High Resource:
  25th Percentile BERTScore: 0.539
  Median BERTScore: 0.57
  Mean BERTScore: 0.557
  SD BERTScore: 0.058
  75th Percentile BERTScore: 0.593
  80th Percentile BERTScore: 0.599
  25th Percentile LLM Rating: 4.0
  Median LLM Rating: 6.0
  Mean LLM Rating: 5.547
  SD LLM Rating: 2.324
  75th Percentile LLM Rating: 8.0
  80th Percentile LLM Rating: 8.0
  Insight Score (novelty): 0.5987652896376643
  Insight Score (feasibility): 0.132201426024955

In [34]:
[cluster_results.iloc[0]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore'], cluster_results.iloc[0]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating']
, cluster_results.iloc[0]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], cluster_results.iloc[0]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']]

[np.float64(0.619),
 np.float64(8.0),
 np.float64(0.2115427970241226),
 np.float64(0.308083464843065)]

In [35]:
i=1
physical_environment_table = [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)']
    ]
]
physical_environment_table

# Define the column labels in the same order as the values in each row.
column_labels = [
    '25th Percentile BERTScore', 
    'Median BERTScore',
    'Mean BERTScore',
    'SD BERTScore',
    '75th Percentile BERTScore',
    '80th Percentile BERTScore',
    '25th Percentile LLM Rating',
    'Median LLM Rating',
    'Mean LLM Rating',
    'SD LLM Rating',
    '75th Percentile LLM Rating',
    '80th Percentile LLM Rating',
    'Insight Score (novelty)', 
    'Insight Score (feasibility)'
]


# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(physical_environment_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # Separator between rows

Row Llamma-3.1 70B Low Resource:
  25th Percentile BERTScore: 0.532
  Median BERTScore: 0.555
  Mean BERTScore: 0.547
  SD BERTScore: 0.047
  75th Percentile BERTScore: 0.577
  80th Percentile BERTScore: 0.58
  25th Percentile LLM Rating: 3.0
  Median LLM Rating: 4.0
  Mean LLM Rating: 4.523
  SD LLM Rating: 2.237
  75th Percentile LLM Rating: 6.0
  80th Percentile LLM Rating: 7.0
  Insight Score (novelty): 0.6024222405725296
  Insight Score (feasibility): 0.1662515964240101
----------------------------------------
Row Llamma-3.1 70B High Resource:
  25th Percentile BERTScore: 0.536
  Median BERTScore: 0.57
  Mean BERTScore: 0.549
  SD BERTScore: 0.062
  75th Percentile BERTScore: 0.587
  80th Percentile BERTScore: 0.591
  25th Percentile LLM Rating: 3.0
  Median LLM Rating: 5.0
  Mean LLM Rating: 5.338
  SD LLM Rating: 2.403
  75th Percentile LLM Rating: 7.75
  80th Percentile LLM Rating: 8.0
  Insight Score (novelty): 0.5751497005988023
  Insight Score (feasibility): 0.21523809523809

In [36]:
[
        cluster_results.iloc[1]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[1]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[1]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[1]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']
    ]

[np.float64(0.618),
 np.float64(8.0),
 np.float64(0.0922953451043338),
 np.float64(0.3796147672552167)]

In [37]:
i=2
tech_soc_engineering_table =  [
    # Llamma-3.1 70B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 70B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 70B High Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B Low Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B Low Resource - Insight Score (feasibility)']
    ],
    # Llamma-3.1 405B High Resource
    [
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Llamma-3.1 405B High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Flash High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Flash High Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro Low Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro Low Resource - Insight Score (feasibility)']
    ],
    # Gemini 1.5 Pro High Resource
    [
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['Gemini 1.5 Pro High Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo Low Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo Low Resource - Insight Score (feasibility)']
    ],
    # GPT-3.5 Turbo High Resource
    [
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-3.5 Turbo High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Mini High Resource
    [
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Mini High Resource - Insight Score (feasibility)']
    ],
    # GPT-4o Low Resource
    [
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o Low Resource - Insight Score (feasibility)']
    ],
    # GPT-4o High Resource
    [
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Median BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Mean BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - SD BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile BERTScore'], 
        cluster_results.iloc[i]['GPT-4o High Resource - 25th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Median LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Mean LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - SD LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 75th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - 80th Percentile LLM Rating'],
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (novelty)'], 
        cluster_results.iloc[i]['GPT-4o High Resource - Insight Score (feasibility)']
    ]
]
tech_soc_engineering_table

# Define the column labels in the same order as the values in each row.
column_labels = [
    '25th Percentile BERTScore', 
    'Median BERTScore',
    'Mean BERTScore',
    'SD BERTScore',
    '75th Percentile BERTScore',
    '80th Percentile BERTScore',
    '25th Percentile LLM Rating',
    'Median LLM Rating',
    'Mean LLM Rating',
    'SD LLM Rating',
    '75th Percentile LLM Rating',
    '80th Percentile LLM Rating',
    'Insight Score (novelty)', 
    'Insight Score (feasibility)'
]

# Assuming 'matrix' is your list of rows
for row_index, row in enumerate(tech_soc_engineering_table):
    print(f"Row {row_labels[row_index]}:")
    for label, value in zip(column_labels, row):
        print(f"  {label}: {value}")
    print("-" * 40)  # S

Row Llamma-3.1 70B Low Resource:
  25th Percentile BERTScore: 0.536
  Median BERTScore: 0.558
  Mean BERTScore: 0.555
  SD BERTScore: 0.034
  75th Percentile BERTScore: 0.576
  80th Percentile BERTScore: 0.583
  25th Percentile LLM Rating: 3.0
  Median LLM Rating: 4.0
  Mean LLM Rating: 4.434
  SD LLM Rating: 2.074
  75th Percentile LLM Rating: 6.0
  80th Percentile LLM Rating: 6.0
  Insight Score (novelty): 0.6192995286851942
  Insight Score (feasibility): 0.195644401105152
----------------------------------------
Row Llamma-3.1 70B High Resource:
  25th Percentile BERTScore: 0.542
  Median BERTScore: 0.566
  Mean BERTScore: 0.561
  SD BERTScore: 0.041
  75th Percentile BERTScore: 0.586
  80th Percentile BERTScore: 0.591
  25th Percentile LLM Rating: 3.0
  Median LLM Rating: 5.0
  Mean LLM Rating: 5.016
  SD LLM Rating: 2.226
  75th Percentile LLM Rating: 7.0
  80th Percentile LLM Rating: 7.0
  Insight Score (novelty): 0.6404680643588494
  Insight Score (feasibility): 0.21203433754454